## PyTorch Tutorial


### Torch Autograd, Variables, Define-by-run & Execution Paradigm

Adapted from
1. http://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#sphx-glr-beginner-blitz-autograd-tutorial-py 
2. http://pytorch.org/docs/master/notes/autograd.html

## Variables : Thin wrappers around tensors to facilitate autograd

Supports almost all operations that can be performed on regular tensors

In [ ]:
import numpy as np


In [ ]:
import torch 
from torch.autograd import Variable

![caption](images/Variable.png)

### Wrap tensors in a Variable

In [ ]:
z = Variable(torch.Tensor(5, 3).uniform_(-1, 1))
print(z)

### Properties of Variables : Requiring gradients, Volatility, Data & Grad

1. You can access the raw tensor through the .data attribute
2. Gradient of the loss w.r.t. this variable is accumulated into .grad.
3. Stay tuned for requires_grad and volatile

In [ ]:
print('Requires Gradient : %s ' % (z.requires_grad))
print('Volatile : %s ' % (z.volatile))
print('Gradient : %s ' % (z.grad))
print(z.data)

In [ ]:
### Operations on Variables
x = Variable(torch.Tensor(5, 3).uniform_(-1, 1))
y = Variable(torch.Tensor(3, 5).uniform_(-1, 1))
# matrix multiplication
z = torch.mm(x, y)
print(z.size())

## Define-by-run Paradigm

The torch autograd package provides automatic differentiation for all operations on Tensors.

PyTorch's autograd is a reverse mode automatic differentiation system.

Backprop is defined by how your code is run, and that every single iteration can be different.

Other frameworks that adopt a similar approach :

1. Chainer - https://github.com/chainer/chainer
2. DyNet - https://github.com/clab/dynet
3. Tensorflow Eager - https://research.googleblog.com/2017/10/eager-execution-imperative-define-by.html

### How autograd encodes execution history


Conceptually, autograd maintains a graph that records all of the operations performed on variables as you execute your operations. This results in a directed acyclic graph whose leaves are the input variables and roots are the output variables. By tracing this graph from roots to leaves, you can automatically compute the gradients using the chain rule.

![caption](images/dynamic_graph.gif)

GIF source: https://github.com/pytorch/pytorch

Internally, autograd represents this graph as a graph of Function objects (really expressions), which can be `apply()` ed to compute the result of evaluating the graph. When computing the forwards pass, autograd simultaneously performs the requested computations and builds up a graph representing the function that computes the gradient (the `.grad_fn` attribute of each Variable is an entry point into this graph). When the forwards pass is completed, we evaluate this graph in the backwards pass to compute the gradients.

In [ ]:
x = Variable(torch.Tensor(5, 3).uniform_(-1, 1))
y = Variable(torch.Tensor(3, 5).uniform_(-1, 1))
z = torch.mm(x, y)
print(z.grad_fn)

An important thing to note is that the graph is recreated from scratch at every iteration, and this is exactly what allows for using arbitrary Python control flow statements, that can change the overall shape and size of the graph at every iteration. You don’t have to encode all possible paths before you launch the training - what you run is what you differentiate.

## Getting gradients : `backward()` & `torch.autograd.grad`

In [ ]:
x = Variable(torch.Tensor(5, 3).uniform_(-1, 1), requires_grad=True)
y = Variable(torch.Tensor(5, 3).uniform_(-1, 1), requires_grad=True)
z = x ** 2 + 3 * y
z.backward(gradient=torch.ones(5, 3))

In [ ]:
# eq computes element-wise equality
torch.eq(x.grad, 2 * x)

In [ ]:
y.grad

In [ ]:
x = Variable(torch.Tensor(5, 3).uniform_(-1, 1), requires_grad=True)
y = Variable(torch.Tensor(5, 3).uniform_(-1, 1), requires_grad=True)
z = x ** 2 + 3 * y
dz_dx = torch.autograd.grad(z, x, grad_outputs=torch.ones(5, 3))
dz_dy = torch.autograd.grad(z, y, grad_outputs=torch.ones(5, 3))

## Define-by-run example

### Common Variable definition

In [ ]:
x = Variable(torch.Tensor(5, 3).uniform_(-1, 1), requires_grad=True)
w = Variable(torch.Tensor(3, 10).uniform_(-1, 1), requires_grad=True)
b = Variable(torch.Tensor(10,).uniform_(-1, 1), requires_grad=True)

### Graph 1 : `wx + b`

In [ ]:
o = torch.matmul(x, w) + b
do_dinputs_1 = torch.autograd.grad(o, [x, w, b], grad_outputs=torch.ones(5, 10))

In [ ]:
print('Gradients of o w.r.t inputs in Graph 1')
print('do/dx : \n\n %s ' % (do_dinputs_1[0]))
print('do/dw : \n\n %s ' % (do_dinputs_1[1]))
print('do/db : \n\n %s ' % (do_dinputs_1[2]))

### Graph 2 : wx / b

In [ ]:
o = torch.matmul(x, w) / b
do_dinputs_2 = torch.autograd.grad(o, [x, w, b], grad_outputs=torch.ones(5, 10))

In [ ]:
print('Gradients of o w.r.t inputs in Graph 2')
print('do/dx : \n %s ' % (do_dinputs_2[0]))
print('do/dw : \n %s ' % (do_dinputs_2[1]))
print('do/db : \n %s ' % (do_dinputs_2[2]))

## Gradient buffers: `.backward()` and `retain_graph=True`

1. Calling `.backward()` clears the current computation graph.
2. Once `.backward()` is called, intermediate variables used in the construction of the graph are removed.
2. This is used implicitly to let PyTorch know when a new graph is to be built for a new minibatch. This is built around the forward and backward pass paradigm.
3. To retain the graph after the backward pass use `loss.backward(retain_graph=True)`. This lets you re-use intermediate variables to potentially compute a secondary loss after the initial gradients are computed. This is useful to implement things like the gradient penalty in WGANs (https://arxiv.org/abs/1704.00028)

In [ ]:
o = torch.mm(x, w) + b
o.backward(torch.ones(5, 10))

### Call backward again -> <font color='red'>This fails</font>

In [ ]:
o = o ** 3
o.backward(torch.ones(5, 10))

###  But with `retain_graph=True`

In [ ]:
o = torch.mm(x, w) + b
o.backward(torch.ones(5, 10), retain_graph=True)

In [ ]:
o = o ** 3
o.backward(torch.ones(5, 10))

## <font color='red'>WARNING:</font> Calling `.backward()` multiple times will accumulate gradients into `.grad` and NOT overwrite them.

## Excluding subgraphs from backward: requires_grad=False, volatile=True & .detach

### `requires_grad=False`

1. If there’s a single input to an operation that requires gradient, its output will also require gradient.

2. Conversely, if all inputs don’t require gradient, the output won’t require it.

3. Backward computation is never performed in the subgraphs, where all Variables didn’t require gradients.

4. This is potentially useful when you have part of a network that is pretrained and not fine-tuned, for example word embeddings or a pretrained imagenet model.

In [ ]:
x = Variable(torch.Tensor(3, 5).uniform_(-1, 1), requires_grad=False)
y = Variable(torch.Tensor(3, 5).uniform_(-1, 1), requires_grad=False)
z = Variable(torch.Tensor(3, 5).uniform_(-1, 1), requires_grad=True)

In [ ]:
o = x + y
print(' o = x + y requires grad ? : %s ' % (o.requires_grad))
o = x + y + z
print(' o = x + y + z requires grad ? : %s ' % (o.requires_grad))

### `volatile=True`

1. If a single input to an operation is volatile, the resulting variable will not have a `grad_fn` and so, the result will not be a node in the computation graph.

2. Conversely, only if all inputs are not volatile, the output will have a `grad_fn` and be included in the computation graph.

3. Volatile is useful when running Variables through your network during inference. Since it is fairly uncommon to go backwards through the network during inference, `.backward()` is rarely invoked. This means graphs are never cleared and hence it is common to run out of memory pretty quickly. Since operations on `volatile` variables are not recorded on the tape and therfore save memory.

In [ ]:
x = Variable(torch.Tensor(3, 5).uniform_(-1, 1), volatile=True)
y = Variable(torch.Tensor(3, 5).uniform_(-1, 1), volatile=True)
z = Variable(torch.Tensor(3, 5).uniform_(-1, 1), requires_grad=True)

In [ ]:
print('Graph : x + y')
o = x + y
print('o.requires_grad : %s ' % (o.requires_grad))
print('o.grad_fn : %s ' % (o.grad_fn))
print('\n\nGraph : x + y + z')
o = x + y + z
print('o.requires_grad : %s ' % (o.requires_grad))
print('o.grad_fn : %s ' % (o.grad_fn))

### `.detach()`

1. It is possible to detach variables from the graph by calling `.detach()`. 
2. This could lead to disconnected graphs. In which case PyTorch will only backpropagate gradients until the point of disconnection.

In [ ]:
x = Variable(torch.Tensor(3, 5).uniform_(-1, 1), requires_grad=True)
y = Variable(torch.Tensor(3, 5).uniform_(-1, 1), requires_grad=True)
z = Variable(torch.Tensor(3, 5).uniform_(-1, 1), requires_grad=True)

![caption](images/detach.png)

In [ ]:
m1 = x + y
m2 = z ** 2
m1 = m1.detach()
m3 = m1 + m2
m3.backward(torch.ones(3, 5))

In [ ]:
print('dm3/dx \n\n %s ' % (x.grad))
print('\ndm3/dy \n\n %s ' % (y.grad))
print('\ndm3/dz \n\n %s ' % (z.grad))

## Gradients w.r.t intermediate variables in the graph

1. By default, PyTorch all gradient computations w.r.t intermediate nodes in the graph are ad-hoc.

2. This is in the interest of saving memory.

3. To compute gradients w.r.t intermediate variables, use `.retain_grad()` or explicitly compute gradients using `torch.autograd.grad`

4. `.retain_grad()` populates the `.grad` attribute of the Variable while `torch.autograd.grad` returns a Variable that contains the gradients.

In [ ]:
x = Variable(torch.Tensor(3, 5).uniform_(-1, 1), requires_grad=True)
y = Variable(torch.Tensor(3, 5).uniform_(-1, 1), requires_grad=True)
z = Variable(torch.Tensor(3, 5).uniform_(-1, 1), requires_grad=True)

In [ ]:
m1 = x + y
m2 = z ** 2
m1.retain_grad()
m2.retain_grad()
m3 = m1 * m2
m3.backward(torch.ones(3, 5))

In [ ]:
print('dm3/dm1 \n\n %s ' % (m1.grad))
print('dm3/dm2 \n\n %s ' % (m2.grad))

### In place operations on variables in a graph

source: http://pytorch.org/docs/master/notes/autograd.html

In place operations are suffixed by `_` ex: `log_`, `uniform_` etc.

1. Supporting in-place operations in autograd is difficult and PyTorch discourages their use in most cases.

2. Autograd’s aggressive buffer freeing and reuse makes it very efficient and there are very few occasions when in-place operations actually lower memory usage by any significant amount. Unless you’re operating under heavy memory pressure, you might never need to use them.

### There are two main reasons that limit the applicability of in-place operations:

(a) Overwriting values required to compute gradients. This is why variables don’t support `log_`. Its gradient formula requires the original input, and while it is possible to recreate it by computing the inverse operation, it is numerically unstable, and requires additional work that often defeats the purpose of using these functions.

(b) Every in-place operation actually requires the implementation to rewrite the computational graph. Out-of-place versions simply allocate new objects and keep references to the old graph, while in-place operations, require changing the creator of all inputs to the Function representing this operation. This can be tricky, especially if there are many Variables that reference the same storage (e.g. created by indexing or transposing), and in-place functions will actually raise an error if the storage of modified inputs is referenced by any other Variable.
In-place correctness checks

## Second and higher order derivatives

### Computing gradients w.r.t gradients

1. `o = xy + z`
2. `l = o + do_dz`

### Practical application of this in WGAN-GP later in the tutorial

In [ ]:
x = Variable(torch.Tensor(5, 3).uniform_(-1, 1), requires_grad=True)
y = Variable(torch.Tensor(3, 5).uniform_(-1, 1), requires_grad=True)
z = Variable(torch.Tensor(5, 5).uniform_(-1, 1), requires_grad=True)

In [ ]:
o = torch.mm(x, y) + z ** 2
# if create_graph=False then the resulting gradient is volatile and cannot be used further to compute a second loss.
do_dz = torch.autograd.grad(o, z, grad_outputs=torch.ones(5, 5), retain_graph=True, create_graph=True)
print('do/dz \n\n : %s ' % (do_dz[0]))
l = o ** 3 + do_dz[0]
dl_dz = torch.autograd.grad(l, z, grad_outputs=torch.ones(5, 5))
print('dl/dz \n\n : %s ' % (dl_dz[0]))